# Data Preparation 

### Linking Anthropology's Data and Archives (LADA)

### AI-Generated Linked Data Evaluation (part I)

In [1]:
import config
import utils
import pandas as pd
import numpy as np
from pathlib import Path
import os
import re

/Users/lucyhavens/Desktop/repos/lada-eval/utils.py:123: SyntaxWarning: invalid escape sequence '\<'
  has_prolog = re.findall("^\<\?xml version=.+ encoding=.+", f_string)
/Users/lucyhavens/Desktop/repos/lada-eval/utils.py:150: SyntaxWarning: invalid escape sequence '\s'
  comments = re.findall("\n\s*\/\/\s*\w.+|\n\s*\/\*\s*.+\s*\*\/|\n\s*#\s*.+", f_string)
/Users/lucyhavens/Desktop/repos/lada-eval/utils.py:157: SyntaxWarning: invalid escape sequence '\w'
  double_quotes = re.findall("""[@\w]+"":[\s\[\{]*"".+""", f_string)
/Users/lucyhavens/Desktop/repos/lada-eval/utils.py:162: SyntaxWarning: invalid escape sequence '\{'
  open_brace = re.findall("\{", f_string)
/Users/lucyhavens/Desktop/repos/lada-eval/utils.py:163: SyntaxWarning: invalid escape sequence '\}'
  close_brace = re.findall("\}", f_string)


Read in CSV data as a pandas DataFrame, where `f` is the name of the data file:

In [36]:
# Uncomment one of the following code lines or replace with your own
# path to the folder (directory) where the data file f is located
# ------------------------------------
# data_dir = config.task1_data
data_dir = config.playgrd1_data
# data_dir = config.playgrd3_data
# data_dir = "path/to/your/data/"

# The name of the data file
# ------------------------------------
# f = "4-H Data Experiment Assignments and Outcomes - Playground Outcomes - Task 1.csv"
f = "4-H Data Experiment Assignments and Outcomes - Outcomes - Task 1.csv"
# f = "4-H Data Experiment Assignments and Outcomes - Playground Outcomes - Task 3.csv"
# f = "your_data_file_name.csv"

In [37]:
df = pd.read_csv(data_dir + f)
print(df.shape)
df.head(10)

(140, 8)


,ID,Filename,Team member Assigned,Metadata record,"Transcription or caption (or link to separate doc, if too long)",Schema.org Record,CIDOC-CRM Record,prompts to chatGPT
0,1,Recognition for Meritorious Service Plaque - S...,Katrina,1. Recognition for Meritorious Service Plaque ...,See Google Link for results,See Google Link for results,See Google Link for results,See Google Link for results
1,2,Criteria for Tenure and Promotion and Examples...,Katrina,2. Criteria for Tenure a,See Google Link for results,See Google Link for results,See Google Link for results,See Google Link for results
2,3,“Turnin’ Timez” original student poems.pdf,Katrina,turnin_timez_dublin_core.xml \n<?xml version='...,turnin_timez_transcription.txt,"{\n ""@context"": ""https://schema.org"",\n ""@ty...","{\n ""@context"": {\n ""crm"": ""http://www.cid...",NaN
3,4,1985_11_03_Agenda_Executive_Committee_Meeting.jpg,Katrina,3. 1985_11_03_Agenda_Executive_Committee_Meeti...,See Google Link for results,See Google Link for results,See Google Link for results,See Google Link for results
4,5,Climbing Up Fun Activites for You and Your Cat...,Katrina,<dc:title>Climbing Up: Fun Activities for You ...,Climbing_Up_OCR_Text.txt,"{\r\n ""@context"": ""https://schema.org"",\r\n ...",@prefix crm: <http://www.cidoc-crm.org/cidoc-c...,NaN
5,6,NAE4-HA On a Fast Track in Kentucky cover20221...,Diana,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<dubli...",Here is the corrected transcription with error...,"{\n ""@context"": ""https://schema.org"",\n ""@ty...","{\n ""@context"": {\n ""@vocab"": ""http://www....",“transcribe this and create a Dublin Core reco...
6,7,FifthLeaf.pdf,Diana,"<metadata xmlns=""http://www.openarchives.org/O...",The document contains more text than initially...,"{\n ""@context"": ""https://schema.org"",\n ""@ty...",Here’s a CIDOC Conceptual Reference Model (CID...,\n“transcribe this entire document exactly and...
7,8,C-3.pdf,Diana,"<dublincore xmlns=""http://purl.org/dc/elements...",NATIONAL 4-H PUBLIC SERVICE SPOTS\nProvided by...,Below is a **Schema.org JSON-LD** record for t...,Here’s a **CIDOC-CRM JSON-LD** record based on...,“transcribe this document and create a Dublin ...
8,9,Explore! columbus center brochure-combined.pdf,Diana,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<dubli...",### Explore! Columbus Center Brochure\n\n---\n...,"{\n ""@context"": ""https://schema.org"",\n ""@ty...","{\n ""@context"": {\n ""@vocab"": ""http://www....",“transcribe this entire document exactly”\n“ma...
9,10,B-6.1.pdf,Diana,<dublin_core>\n <dc:title>4-H for Youth for A...,4-H\nFOR YOUTH\nFOR AMERICA This suggests a pr...,"{\n ""@context"": ""https://schema.org"",\n ""@ty...","{\n ""@context"": {\n ""crm"": ""http://www.cid...",“transcribe and describe this and create a Dub...


Remove empty columns from the DataFrame:

In [38]:
print(df.shape)
df = df.dropna(axis=1, how='all')
print(df.shape)
df.head()

(140, 8)
(140, 8)


,ID,Filename,Team member Assigned,Metadata record,"Transcription or caption (or link to separate doc, if too long)",Schema.org Record,CIDOC-CRM Record,prompts to chatGPT
0,1,Recognition for Meritorious Service Plaque - S...,Katrina,1. Recognition for Meritorious Service Plaque ...,See Google Link for results,See Google Link for results,See Google Link for results,See Google Link for results
1,2,Criteria for Tenure and Promotion and Examples...,Katrina,2. Criteria for Tenure a,See Google Link for results,See Google Link for results,See Google Link for results,See Google Link for results
2,3,“Turnin’ Timez” original student poems.pdf,Katrina,turnin_timez_dublin_core.xml \n<?xml version='...,turnin_timez_transcription.txt,"{\n ""@context"": ""https://schema.org"",\n ""@ty...","{\n ""@context"": {\n ""crm"": ""http://www.cid...",NaN
3,4,1985_11_03_Agenda_Executive_Committee_Meeting.jpg,Katrina,3. 1985_11_03_Agenda_Executive_Committee_Meeti...,See Google Link for results,See Google Link for results,See Google Link for results,See Google Link for results
4,5,Climbing Up Fun Activites for You and Your Cat...,Katrina,<dc:title>Climbing Up: Fun Activities for You ...,Climbing_Up_OCR_Text.txt,"{\r\n ""@context"": ""https://schema.org"",\r\n ...",@prefix crm: <http://www.cidoc-crm.org/cidoc-c...,NaN


Remove rows without a metadata record from the DataFrame:

In [39]:
df.dropna(subset=["Metadata record"], inplace=True)
print(df.shape)
df.tail()

(125, 8)


,ID,Filename,Team member Assigned,Metadata record,"Transcription or caption (or link to separate doc, if too long)",Schema.org Record,CIDOC-CRM Record,prompts to chatGPT
135,136,0119_0000_Development-Photos,Desmond,UNABLE TO EXTRACT TEXT,UNABLE TO EXTRACT TEXT,UNABLE TO EXTRACT TEXT,UNABLE TO EXTRACT TEXT,NaN
136,137,0063_1988_International-Symposium,Desmond,Title: 4-H USA International Programs\r\nCreat...,I can see that this is a comprehensive speech ...,"{\r\n ""@context"": ""https://schema.org"",\r\n ...",@prefix crm: <http://www.cidoc-crm.org/cidoc-c...,NaN
137,138,0096_1985_Communications-Newsbreak,Desmond,UNABLE TO EXTRACT TEXT,UNABLE TO EXTRACT TEXT,UNABLE TO EXTRACT TEXT,UNABLE TO EXTRACT TEXT,NaN
138,139,0076_1982_History-Canada,Desmond,UNABLE TO EXTRACT TEXT,UNABLE TO EXTRACT TEXT,UNABLE TO EXTRACT TEXT,UNABLE TO EXTRACT TEXT,NaN
139,140,0025_1988_Cooperative-Extension-Booklet,Desmond,UNABLE TO EXTRACT TEXT,UNABLE TO EXTRACT TEXT,UNABLE TO EXTRACT TEXT,UNABLE TO EXTRACT TEXT,NaN


If there's no column with unique identifiers per row, make one:

In [40]:
col_list = list(df.columns)
identifier_col = ""
for col in col_list:
    if (col.lower() == "id") or (col.lower() == "identifier") or ("_id" in col.lower()):
        # Check that the existing IDs are, in fact, unique to each row
        existing_ids = list(df[col])
        if len(existing_ids) == len(set(existing_ids)):
            identifier_col = col
if len(identifier_col) == 0:
    # Create a unique identifier per row
    new_ids = list(range(0, df.shape[0]))
    df.insert(0, "id", new_ids)

In [41]:
print("Rows:", df.shape[0])
print("Columns:", df.shape[1])

Rows: 125
Columns: 8


Replace the column names for clarity, consistency, and conciseness:

In [42]:
# Note: if your data header is different, replace the strings of column names before
# each colon with the column names of your data (i.e., "ID" might become "identifier").
# The strings after the colon are the new column names that the code below will expect.
if len(identifier_col) > 0:
        df.rename(columns={
        identifier_col:"id",                  # Unique identifier for each row
        "Filename":"filename", 
        "Metadata record": "dc_record",       # Dublin Core metadata record
        #"Transcription or caption (or link to separate doc, if too long)":"transcription_or_caption",
        "Schema.org Record":"sdo_record",     # Schema.org metadata record
        "CIDOC-CRM Record":"cidoccrm_record"  # CIDOC-CRM metadata record
        }, inplace=True)
else:
    df.rename(columns={
        "Filename":"filename", 
        "Metadata record": "dc_record",       # Dublin Core metadata record
        #"Transcription or caption (or link to separate doc, if too long)":"transcription_or_caption",
        "Schema.org Record":"sdo_record",     # Schema.org metadata record
        "CIDOC-CRM Record":"cidoccrm_record"  # CIDOC-CRM metadata record
        }, inplace=True)
    
df.head()


,id,filename,Team member Assigned,dc_record,"Transcription or caption (or link to separate doc, if too long)",sdo_record,cidoccrm_record,prompts to chatGPT
0,1,Recognition for Meritorious Service Plaque - S...,Katrina,1. Recognition for Meritorious Service Plaque ...,See Google Link for results,See Google Link for results,See Google Link for results,See Google Link for results
1,2,Criteria for Tenure and Promotion and Examples...,Katrina,2. Criteria for Tenure a,See Google Link for results,See Google Link for results,See Google Link for results,See Google Link for results
2,3,“Turnin’ Timez” original student poems.pdf,Katrina,turnin_timez_dublin_core.xml \n<?xml version='...,turnin_timez_transcription.txt,"{\n ""@context"": ""https://schema.org"",\n ""@ty...","{\n ""@context"": {\n ""crm"": ""http://www.cid...",NaN
3,4,1985_11_03_Agenda_Executive_Committee_Meeting.jpg,Katrina,3. 1985_11_03_Agenda_Executive_Committee_Meeti...,See Google Link for results,See Google Link for results,See Google Link for results,See Google Link for results
4,5,Climbing Up Fun Activites for You and Your Cat...,Katrina,<dc:title>Climbing Up: Fun Activities for You ...,Climbing_Up_OCR_Text.txt,"{\r\n ""@context"": ""https://schema.org"",\r\n ...",@prefix crm: <http://www.cidoc-crm.org/cidoc-c...,NaN


Create a directory to store the cleaner version of the data:

In [43]:
data_dir = data_dir + "cleaned/"
Path(data_dir).mkdir(parents=True, exist_ok=True)
df.to_csv(data_dir + f)

In [44]:
record_ids = list(df["id"])
print("Total IDs (rows):", len(record_ids))
print("Sample ID:", record_ids[2])

Total IDs (rows): 125
Sample ID: 3


### Dublin Core
Write the [Dublin Core](https://www.dublincore.org) (DC) records as XML files.

In [58]:
# Remove rows with an empty value in the dc_record column
df_dc = df[df["dc_record"].notna()]
dc_records_ids = list(df_dc["id"])

In [59]:
dc_records = list(df_dc["dc_record"])
print(dc_records[2])

<metadata xmlns:dc="http://purl.org/dc/elements/1.1/">
  <dc:title>Derrick A. Bell, Jr. Papers</dc:title>
  <dc:creator>Bell, Derrick A., 1930-2011</dc:creator>
  <dc:contributor>New York University Archives</dc:contributor>
  <dc:date>1922-2011</dc:date>
  <dc:description>
    The papers document the personal and professional life of Derrick Bell, a pioneering civil rights attorney, legal scholar, and professor. The collection includes correspondence, legal documents, writings, teaching materials, and photographs, with significant content related to civil rights, critical race theory, and higher education.
  </dc:description>
  <dc:language>en</dc:language>
  <dc:identifier>MC.138</dc:identifier>
  <dc:publisher>New York University Archives</dc:publisher>
  <dc:rights>
    FERPA restrictions may apply. Contact NYU Archives for specific access and use conditions.
  </dc:rights>
  <dc:format>189 boxes of textual and born-digital materials</dc:format>
  <dc:type>Archival collection</dc:t

In [60]:
dc_path = data_dir+"dublin_core/"
Path(dc_path).mkdir(parents=True, exist_ok=True)

In [61]:
print(dc_path)

data/data_playground_task3/cleaned/dublin_core/


In [62]:
utils.write_xml(dc_records_ids, dc_records, dc_path, "dc_record_", ".xml")
utils.write_xml(dc_records_ids, dc_records, dc_path, "dc_record_", ".txt")

Wrote dc_record_006.xml!
Wrote dc_record_007.xml!
Wrote dc_record_008.xml!
Wrote dc_record_009.xml!
Wrote dc_record_010.xml!
Wrote dc_record_011.xml!
Wrote dc_record_012.xml!
Wrote dc_record_013.xml!
Wrote dc_record_014.xml!
Wrote dc_record_015.xml!
Wrote dc_record_016.xml!
Wrote dc_record_017.xml!
Wrote dc_record_018.xml!
Wrote dc_record_019.xml!
Wrote dc_record_020.xml!
Wrote dc_record_021.xml!
Wrote dc_record_022.xml!
Wrote dc_record_023.xml!
Wrote dc_record_024.xml!
Wrote dc_record_025.xml!
Wrote dc_record_006.txt!
Wrote dc_record_007.txt!
Wrote dc_record_008.txt!
Wrote dc_record_009.txt!
Wrote dc_record_010.txt!
Wrote dc_record_011.txt!
Wrote dc_record_012.txt!
Wrote dc_record_013.txt!
Wrote dc_record_014.txt!
Wrote dc_record_015.txt!
Wrote dc_record_016.txt!
Wrote dc_record_017.txt!
Wrote dc_record_018.txt!
Wrote dc_record_019.txt!
Wrote dc_record_020.txt!
Wrote dc_record_021.txt!
Wrote dc_record_022.txt!
Wrote dc_record_023.txt!
Wrote dc_record_024.txt!
Wrote dc_record_025.txt!


**Note:** There is inconsistency in the DC record formatting!  For example:

```
<?xml version='1.0' encoding='utf-8'?>
<dublin_core><dc element="title">Turnin' Timez: Original Student Poems</dc>
    ...
</dublin_core>
```
---
```
<?xml version="1.0"?>
<metadata xmlns:dc="http://purl.org/dc/elements/1.1/">
<dc:title>National 4-H Center Major Pledges, Contributions, and Grants</dc:title>
    ...
</metadata>
```
---
```
<?xml version="1.0" encoding="UTF-8"?>
<metadata xmlns:dcterms="http://purl.org/dc/terms/" xmlns:dc="http://purl.org/dc/elements/1.1/">
<dc:title>4-H National Youth Science Day</dc:title>
    ...
</metadata>
```
---
```
<dc:title>Climbing Up: Fun Activities for You and Your Cat</dc:title>
   ...
<dc:rights>Unknown</dc:rights>
```

Also note that [DCMI documentation](https://www.dublincore.org/specifications/dublin-core/dcmi-terms/) encourages the use of http://purl.org/dc/terms/ over http://purl.org/dc/elements/1.1/.

### Schema.org
Write the [Schema.org](https://schema.org) records as JSON-LD files.

In [45]:
# Remove rows with an empty value in the sdo_record column
df_sdo = df[df["sdo_record"].notna()]
sdo_records_ids = list(df_sdo["id"])

In [46]:
sdo_path = data_dir+"schema_org/"
Path(sdo_path).mkdir(parents=True, exist_ok=True)

In [47]:
sdo_records = list(df_sdo["sdo_record"])
print(sdo_records[2])

{
  "@context": "https://schema.org",
  "@type": "CreativeWork",
  "name": "Turnin' Timez: Original Student Poems",
  "author": "Various Authors (Students)",
  "keywords": "Poetry, Student Creative Writing, Reflections",
  "description": "A collection of original poems written by students, reflecting on various themes such as identity, change, and personal growth.",
  "publisher": "Unknown",
  "datePublished": "2024-12-04",
  "encodingFormat": "PDF",
  "inLanguage": "English",
  "identifier": "urn:uuid:turnin-timez-001"
}


In [48]:
utils.write_json(sdo_records_ids, sdo_records, sdo_path, "sdo_record_", ".json")
utils.write_json(sdo_records_ids, sdo_records, sdo_path, "sdo_record_", ".txt")

Wrote sdo_record_003.json!
Wrote sdo_record_005.json!
Wrote sdo_record_006.json!
Wrote sdo_record_007.json!
Wrote sdo_record_008.json!
Wrote sdo_record_009.json!
Wrote sdo_record_010.json!
Wrote sdo_record_011.json!
Wrote sdo_record_013.json!
Wrote sdo_record_014.json!
Wrote sdo_record_016.json!
Wrote sdo_record_017.json!
Wrote sdo_record_018.json!
Wrote sdo_record_019.json!
Wrote sdo_record_020.json!
Wrote sdo_record_031.json!
Wrote sdo_record_032.json!
Wrote sdo_record_033.json!
Wrote sdo_record_034.json!
Wrote sdo_record_038.json!
Wrote sdo_record_043.json!
Wrote sdo_record_044.json!
Wrote sdo_record_045.json!
Wrote sdo_record_046.json!
Wrote sdo_record_047.json!
Wrote sdo_record_048.json!
Wrote sdo_record_049.json!
Wrote sdo_record_050.json!
Wrote sdo_record_052.json!
Wrote sdo_record_053.json!
Wrote sdo_record_056.json!
Wrote sdo_record_057.json!
Wrote sdo_record_058.json!
Wrote sdo_record_059.json!
Wrote sdo_record_062.json!
Wrote sdo_record_063.json!
Wrote sdo_record_064.json!
W

### CIDOC-CRM
Write the [CIDOC-CRM](https://cidoc-crm.org) records as JSON-LD files.

In [49]:
# Remove rows with an empty value in the cidoccrm_record column
df_cidoc = df[df["cidoccrm_record"].notna()]
cidoc_records_ids = list(df_cidoc["id"])

In [50]:
cidoc_path = data_dir+"cidoc_crm/"
Path(cidoc_path).mkdir(parents=True, exist_ok=True)

In [51]:
cidoc_records = list(df_cidoc["cidoccrm_record"])
print(cidoc_records[2])

{
  "@context": {
    "crm": "http://www.cidoc-crm.org/cidoc-crm/",
    "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
    "xsd": "http://www.w3.org/2001/XMLSchema#"
  },
  "@type": "crm:E73_Information_Object",
  "crm:P102_has_title": {
    "@value": "Turnin' Timez: Original Student Poems",
    "@language": "en"
  },
  "crm:P94_has_created": {
    "@type": "crm:E21_Person",
    "rdfs:label": "Various Authors (Students)"
  },
  "crm:P3_has_note": {
    "@value": "A collection of original poems written by students, reflecting on various themes such as identity, change, and personal growth.",
    "@language": "en"
  },
  "crm:P4_has_time-span": {
    "@type": "crm:E52_Time-Span",
    "crm:P82_at_some_time_within": {
      "@value": "2024-12-04",
      "@type": "xsd:date"
    }
  },
  "crm:P108_has_produced": {
    "@type": "crm:E40_Legal_Body",
    "rdfs:label": "Unknown"
  },
  "crm:P72_has_language": {
    "@type": "crm:E56_Language",
    "rdfs:label": "English"
  },
  "crm:P1_is_ide

In [52]:
utils.write_json(cidoc_records_ids, cidoc_records, cidoc_path, "cidoccrm_record_", ".json")
utils.write_json(cidoc_records_ids, cidoc_records, cidoc_path, "cidoccrm_record_", ".txt")

Wrote cidoccrm_record_003.json!
Wrote cidoccrm_record_006.json!
Wrote cidoccrm_record_007.json!
Wrote cidoccrm_record_008.json!
Wrote cidoccrm_record_009.json!
Wrote cidoccrm_record_010.json!
Wrote cidoccrm_record_011.json!
Wrote cidoccrm_record_013.json!
Wrote cidoccrm_record_014.json!
Wrote cidoccrm_record_016.json!
Wrote cidoccrm_record_017.json!
Wrote cidoccrm_record_018.json!
Wrote cidoccrm_record_019.json!
Wrote cidoccrm_record_020.json!
Wrote cidoccrm_record_043.json!
Wrote cidoccrm_record_044.json!
Wrote cidoccrm_record_045.json!
Wrote cidoccrm_record_046.json!
Wrote cidoccrm_record_047.json!
Wrote cidoccrm_record_048.json!
Wrote cidoccrm_record_049.json!
Wrote cidoccrm_record_050.json!
Wrote cidoccrm_record_052.json!
Wrote cidoccrm_record_053.json!
Wrote cidoccrm_record_056.json!
Wrote cidoccrm_record_057.json!
Wrote cidoccrm_record_058.json!
Wrote cidoccrm_record_059.json!
Wrote cidoccrm_record_061.json!
Wrote cidoccrm_record_062.json!
Wrote cidoccrm_record_063.json!
Wrote ci

Wrote cidoccrm_record_047.txt!
Wrote cidoccrm_record_048.txt!
Wrote cidoccrm_record_049.txt!
Wrote cidoccrm_record_050.txt!
Wrote cidoccrm_record_052.txt!
Wrote cidoccrm_record_053.txt!
Wrote cidoccrm_record_056.txt!
Wrote cidoccrm_record_057.txt!
Wrote cidoccrm_record_058.txt!
Wrote cidoccrm_record_059.txt!
Wrote cidoccrm_record_061.txt!
Wrote cidoccrm_record_062.txt!
Wrote cidoccrm_record_063.txt!
Wrote cidoccrm_record_064.txt!
Wrote cidoccrm_record_070.txt!
Wrote cidoccrm_record_071.txt!
Wrote cidoccrm_record_074.txt!
Wrote cidoccrm_record_075.txt!
Wrote cidoccrm_record_076.txt!
Wrote cidoccrm_record_077.txt!
Wrote cidoccrm_record_078.txt!
Wrote cidoccrm_record_079.txt!
Wrote cidoccrm_record_080.txt!
Wrote cidoccrm_record_081.txt!
Wrote cidoccrm_record_082.txt!
Wrote cidoccrm_record_083.txt!
Wrote cidoccrm_record_084.txt!
Wrote cidoccrm_record_085.txt!
Wrote cidoccrm_record_086.txt!
Wrote cidoccrm_record_088.txt!
Wrote cidoccrm_record_089.txt!
Wrote cidoccrm_record_090.txt!
Wrote ci